# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import requests
import json
import os
from azureml.core import Workspace, Datastore, Dataset, Experiment, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [11]:
# Read and clean the data

raw_data = pd.read_csv('data-unbalanced.csv')
raw_data = raw_data.dropna()
raw_data = raw_data.drop("WO-MRR NUM", axis=1)
raw_data = raw_data.drop("Has MRR", axis=1)
raw_data = pd.get_dummies(raw_data, columns=['PROD_CDE','PREP ASSOCIATE', 'FURN'])
print(raw_data.shape)

(26038, 91)


In [3]:
# Create a new Dataset from the raw_data
ws = Workspace.from_config()
datastore = Datastore.get(ws, 'workspaceblobstore')

dataset = Dataset.Tabular.register_pandas_dataframe(raw_data, datastore ,"Sinter", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/345c97f0-159c-49df-aed0-0c90c2d5f86d/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [4]:
# Register the dataset
dataset = dataset.register(workspace = ws, name='Sinter_dataset', description='Sinter scrap dataset')

In [5]:
# choose a name for experiment
experiment_name = 'Sinter_AML'

experiment=Experiment(ws, experiment_name)

# Create a Copute target

In [7]:
cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HAS SCRAP AT SINTER",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on ML-Cluster with default configuration
Running on remote compute: ML-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Sinter_AML,AutoML_4505acab-d558-4f91-bb55-942f198dff5a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Dataset                          |Row counts                       |Percentage                       |
+=================================+=================================+===============

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [27]:
best_run, best_model = remote_run.get_output()

best_model_metrics = best_run.get_metrics()


In [35]:
# Display the metrics of the best model

metrics = pd.DataFrame.from_dict(best_model_metrics, orient='index', columns=['metric value'])
metrics

,metric value
weighted_accuracy,0.99
AUC_micro,0.95
recall_score_micro,0.91
log_loss,0.31
precision_score_macro,0.71
norm_macro_recall,0.01
precision_score_weighted,0.87
accuracy,0.91
average_precision_score_micro,0.94
recall_score_weighted,0.91


In [89]:
#TODO: Save the best model

aml_model = remote_run.register_model(model_name='AML_Best')

print(aml_model.name)

AML_Best


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [90]:
# Define inference config

env = best_run.get_environment()
dummy_inference_config = InferenceConfig(
    environment= env,
    entry_script="./source_dir/score.py"
)


# Define a deployment configuration

deploy_config = LocalWebservice.deploy_configuration(port=9000)

In [91]:
# Deploy the model

service = Model.deploy(
    ws,
    "myservice",
    [aml_model],
    dummy_inference_config,
    deploy_config
)

service.wait_for_deployment(show_output=True)

ERROR:azureml._restclient.models_client:ModelNotFound: Model with id AutoML9e50608fd24:1 not found in provided workspace

ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_4d4668f442e87b2baa899f119bc648ab
 ---> f1f4c52d7f98
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 8416e671f196
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImEwYTc2YmFkLTExYTEtNGEyZC05ODg3LTk3YTI5MTIyYzhlZCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MTIzNyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MTIzNyIsIndvcmtzcGFjZUlkIjoiZTZlOTNjMjgtMTczYi00YzlmLWE3ODQtNjNlMjJiZGQyMmI0In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 842ab13ecee3
 ---> 78c941ca6e83
Step 4/5 : RUN mv '/var/azureml-app/tmpx_lat_xw.py' /var/azureml-app/main.py
 ---> Running in fefc985328b9
 ---> cba812e6938e
Step 5/5 : 

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [69]:
uri = service.scoring_uri
requests.get("http://localhost:9000")
headers = {"Content-Type":"application/json"}

data = {
    "query": "what color is the fox",
    "context": "the quick..."
}

data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)

print(response.json())

test is {'query': 'what color is the fox', 'context': 'the quick...'}


TODO: In the cell below, print the logs of the web service and delete the service

In [92]:
model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'AutoML_best')

TypeError: expected str, bytes or os.PathLike object, not NoneType

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
